In [3]:
import json
import os 

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [9]:
from openai import OpenAI
client = OpenAI(api_key="sk-***")

In [10]:
assistant = client.beta.assistants.create(
    name = "Finance Insight Analyst",
    instructions = "You are a helpful  financial analyst expert and your are tailored for in-depth SEC 10-Q filings analysis, focusing on management discussions and financial results.",
    tools = [{"type":"code_interpreter"}, {"type": "retrieval"}],
    model = "gpt-4-1106-preview"
)

In [11]:
show_json(assistant)

{'id': 'asst_2xRfTAHL6ck2kZj9wVCGTMPF',
 'created_at': 1702698563,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a helpful  financial analyst expert and your are tailored for in-depth SEC 10-Q filings analysis, focusing on management discussions and financial results.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Finance Insight Analyst',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'retrieval'}]}

In [12]:
# FINANCE_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."
FINANCE_ASSISTANT_ID = "asst_2xRfTAHL6ck2kZj9wVCGTMPF"

In [16]:
def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


import time
# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [17]:
thread1 = client.beta.threads.create()

In [18]:
# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

In [19]:
run1 = submit_message(FINANCE_ASSISTANT_ID, thread1, "What is SEC Q-10 filing?")
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Messages
user: What is SEC Q-10 filing?
assistant: The SEC Form 10-Q is a comprehensive report of financial performance that must be submitted quarterly by all public companies to the Securities and Exchange Commission (SEC) in the United States. The 10-Q is generally shorter than the annual Form 10-K report and provides less detail, but it offers a continuous view of a company's financial position during the year.

The Form 10-Q includes:

1. Unaudited financial statements: These include the balance sheet, income statement, statement of cash flows, and statement of stockholders' equity. Although unaudited, the statements must conform to Generally Accepted Accounting Principles (GAAP) and provide insights into the company's financial health.

2. Management's Discussion and Analysis of Financial Condition and Results of Operations (MD&A): Here the company’s management discusses the financial results and condition of the company, giving investors a narrative explanation of the operatin

In [20]:
# Upload the file
file = client.files.create(
    file=open(
        "Documents/tesla-q3.pdf",
        "rb",
    ),
    purpose="assistants",
)
# Update Assistant
assistant = client.beta.assistants.update(
    FINANCE_ASSISTANT_ID,
    file_ids=[file.id],
)
show_json(assistant)

{'id': 'asst_2xRfTAHL6ck2kZj9wVCGTMPF',
 'created_at': 1702698563,
 'description': None,
 'file_ids': ['file-Dhh0y3AsgFxlwWEtvU2VAMi7'],
 'instructions': 'You are a helpful  financial analyst expert and your are tailored for in-depth SEC 10-Q filings analysis, focusing on management discussions and financial results.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Finance Insight Analyst',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'retrieval'}]}

In [21]:
thread = client.beta.threads.create()
run = submit_message(FINANCE_ASSISTANT_ID, thread, "Summarize the management discussion in this 10K filing and list key takeaways from this financial result")
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: Summarize the management discussion in this 10K filing and list key takeaways from this financial result
assistant: The management discussion from NVIDIA's 10-Q filing provides a comprehensive view of their financial condition, recent developments, future objectives, challenges, and performance across specific segments. The document focuses on forward-looking statements, risks, and factors affecting performance, and it presents detailed financial results for the relevant fiscal periods. Here is a summarized analysis covering the key discussions and financial results:

**Management Discussion:**
1. **Company Overview**: NVIDIA, founded in 1993, is a pioneer in accelerated computing and invented the GPU in 1999, which has driven growth in PC gaming, modern AI, and digitalization across various markets.

2. **Segments**: NVIDIA has two operating segments: "Compute & Networking" and "Graphics".

3. **Financial Highlights**:
   - Q3 Fiscal Year 2024 revenue was $18.12 billi

In [23]:
thread2 = client.beta.threads.create()
run2 = submit_message(FINANCE_ASSISTANT_ID, thread2, "what are the risks related to acquisitionst")
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

# Messages
user: what are the risks related to acquisitionst
assistant: Risks related to acquisitions as mentioned in the document are extensive and include but are not limited to:

1. **Integrating Challenges**: Difficulty in integrating the technology systems, products, policies, processes or operations, and integrating and retaining the employees, including key personnel of the acquired business.

2. **Financial Impact**: Assumption of liabilities and incurring expenses like amortization, impairment charges to goodwill, or write-downs of acquired assets.

3. **Accounting and Reporting Integration**: Challenges in integrating accounting, forecasting, controls procedures, and aligning reporting cycles.

4. **Operational Coordination**: Difficulties in coordinating and integrating operations, particularly in countries where the company does not already operate.

5. **Regulatory Approvals and Impacts**: Potential stock price impacts, fines, fees, or reputation harm if the company is una

In [24]:
run = submit_message(FINANCE_ASSISTANT_ID, thread, "Can you extract the financial summary data et put it in CSV")
run = wait_on_run(run, thread)
messages = get_response(thread)

In [25]:
file_path = messages.data[-1].content[0].text.annotations[0].file_path.file_id

file_name = client.files.with_raw_response.retrieve_content(file_path)
output_file_name = "data.csv"
with open(output_file_name, "wb") as file:
    file.write(file_name.content)

In [26]:
import pandas as pd
df = pd.read_csv("data.csv")
df.head(12)

,Three Months Ended,Revenue ($ in millions),Gross Margin (%),Operating Expenses ($ in millions),Operating Income ($ in millions),Net Income ($ in millions),Net Income per Diluted Share ($),Quarter-over-Quarter Change (%),Year-over-Year Change (%)
0,"October 29, 2023",18120,74.0,2983,10417,9243,3.71,-,-
1,"July 30, 2023",13507,70.1,2662,6800,6188,2.48,34,50
2,"October 30, 2022",5931,53.6,2576,601,680,0.27,206,1274
